# Analyze and Visualize Results

In [1]:
import sys, os, logging
import json
from pathlib import Path
from typing import List, Tuple, Union, Dict, Optional
from multiprocessing import Process, Pool
import psutil
import base64
import asyncio
import cv2
import imageio as iio
import numpy as np
from IPython.core.display import HTML
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq, fftn, fftshift

from turbx import REPO_PATH, log
from turbx.vis import (calc_tfpnr, calc_tdr, calc_frames_removed, label_to_per_frame_list)
from turbx.metrics import (
    calc_box_area,
    boxes_to_binary,
    target_detection_rate,
    tfpnr,
    safe_division
)

[DEBUG] 11/28/2022 04:59:14PM: MainProcess: __init__.py - Loaded logging config file: /Users/nowa201/Code/fish_detector/turbx/src/turbx/logging.yaml


## Load experiment output

In [5]:
base_path = Path(f"{REPO_PATH}/notebooks/outputs/no-tracklets")

## Calculate metrics on experiment data

In [6]:
# calculate per video scores

# Things we want per run,
# 1) mAR
# 2) mAP
# 3) F1
# 4) TDR - if this is not 1 then we can adjust this param search
# TDR can be tought of as the notification rate, we really do not want to miss a notification

all_neg_dets= 0
all_neg_lab= 0
all_frames =0


maxF1 = 0
maxAP = 0
maxAR = 0
maxFramesRemoved = 0
APs = []
ARs = []
F1s = []
FRs = []

#print("Per frame:")
#print("id:, FPR, FNR, Target Detection Rate, % Frames Removed, % Neg Frames Removed")

# accumulate run directories
subdirs = next(os.walk(str(base_path)))[1]
# iterate through each run, and calculate stats
for i, param_set in enumerate(subdirs):
    param_set_path = Path(base_path/param_set)
    param_set_results = []
    # load every json for that run and append to param_set_results
    for f_name in param_set_path.glob("**/*.json"):
        with open(f_name, 'r') as f:
            params = json.load(f)
            param_set_results.append((params, param_set))
    tp = 0
    fp = 0
    fn = 0
    all_neg_dets = 0
    all_neg_lab = 0
    all_frames = 0

    # for each video
    for result, param_set_id in param_set_results:
        # get per video statistics
        binary_label, binary_pred, tfpnr_dict = calc_tfpnr(result['label'], result['prediction'], show=False, save=False)
        unique_targs, det_targs, tdr = calc_tdr(result['label'], result['prediction'])
        _, n_pos_dets, n_neg_dets = calc_frames_removed(result['prediction'])
        _, n_pos_lab, n_neg_lab = calc_frames_removed(label_to_per_frame_list(result['label']))
        perc_frames_removed = 100*(n_neg_dets / len(binary_pred))
        perc_neg_frame_removed = 100*(n_neg_dets / n_neg_lab)
        # TODO: weight videos equally, or weight by frame? -> weight by frame
        # Blerim is adding up all then dividing later -> fair, is by frame
        all_neg_dets += n_neg_dets
        all_neg_lab += n_neg_lab
        all_frames += len(binary_pred)
        tp += tfpnr_dict['tp']
        fp += tfpnr_dict['fp']
        fn += tfpnr_dict['fn']
    
    param_set_AP = tp/(tp+fp)
    param_set_AR = tp/(tp+fn)
    #param_set_AP = safe_division(tp, tp+fp, 1.0)
    #param_set_AR = safe_division(tp, tp+fn, 1.0)
    param_set_F1 = 2/((1/param_set_AR) + (1/param_set_AP))
    APs.append(param_set_AP)
    ARs.append(param_set_AR)
    F1s.append(param_set_F1)

    if(param_set_AP > maxAP):
        maxAP = param_set_AP
        maxAPParamID = param_set
        maxAPParamIDX = i
    if(param_set_AR > maxAR):
        maxAR = param_set_AR
        maxARParamID = param_set
        maxARParamIDX = i
    if(param_set_F1 > maxF1):
        maxF1 = param_set_F1
        maxF1ParamID = param_set
        maxF1ParamIDX = i
    
    #print(param_set_AP)
    perc_all_frames = 100*(all_neg_dets/all_frames)
    perc_tn_frames = 100*(all_neg_lab/all_frames)
    FRs.append(perc_all_frames)
    # calculate best frame remover
    if(perc_all_frames > maxFramesRemoved):
        maxFR = perc_all_frames
        maxFRParamID = param_set
        maxFRParamIDX = i
    
    
    #print(f"\n mean false negative rate: {mean_fnr:.2f}")
    #print(f"\n% of all frames removed: {perc_all_frames:.2f}")
    #print(f"\n% of all frames that are empty: {perc_tn_frames:.2f}")
print(f"Best Average Precision:\nParam Set: {maxAPParamID},\tAP: {APs[maxAPParamIDX]:.2f},\tAR: {ARs[maxAPParamIDX]:.2f},\tF1: {F1s[maxAPParamIDX]:.2f},\tFR: {FRs[maxAPParamIDX]:.2f}")
print(f"Best Average Recall:\nParam Set: {maxARParamID},\tAP: {APs[maxARParamIDX]:.2f},\tAR: {ARs[maxARParamIDX]:.2f},\tF1: {F1s[maxARParamIDX]:.2f},\tFR: {FRs[maxARParamIDX]:.2f}")
print(f"Best F1:\nParam Set: {maxF1ParamID},\tAP: {APs[maxF1ParamIDX]:.2f},\tAR: {ARs[maxF1ParamIDX]:.2f},\tF1: {F1s[maxF1ParamIDX]:.2f},\tFR: {FRs[maxF1ParamIDX]:.2f}")
print(f"Best Frame Remover:\nParam Set: {maxFRParamID},\tAP: {APs[maxFRParamIDX]:.2f},\tAR: {ARs[maxFRParamIDX]:.2f},\tF1: {F1s[maxFRParamIDX]:.2f},\tFR: {FRs[maxFRParamIDX]:.2f}")

Best Average Precision:
Param Set: 187,	AP: 0.40,	AR: 0.34,	F1: 0.37,	FR: 86.60
Best Average Recall:
Param Set: 57,	AP: 0.20,	AR: 0.88,	F1: 0.33,	FR: 30.88
Best F1:
Param Set: 168,	AP: 0.35,	AR: 0.49,	F1: 0.41,	FR: 77.49
Best Frame Remover:
Param Set: 25,	AP: 0.28,	AR: 0.58,	F1: 0.37,	FR: 66.12


No tracklet results
```
Best Average Precision:
Param Set: 187,	AP: 0.40,	AR: 0.34,	F1: 0.37,	FR: 86.60
Best Average Recall:
Param Set: 57,	AP: 0.20,	AR: 0.88,	F1: 0.33,	FR: 30.88
Best F1:
Param Set: 168,	AP: 0.35,	AR: 0.49,	F1: 0.41,	FR: 77.49
Best Frame Remover:
Param Set: 25,	AP: 0.28,	AR: 0.58,	F1: 0.37,	FR: 66.12
```

With tracklet results
```
Best Average Precision:
Param Set: 88,	AP: 0.34,	AR: 0.63,	F1: 0.44,	FR: 70.04
Best Average Recall:
Param Set: 20,	AP: 0.19,	AR: 0.94,	F1: 0.32,	FR: 22.21
Best F1:
Param Set: 88,	AP: 0.34,	AR: 0.63,	F1: 0.44,	FR: 70.04
Best Frame Remover:
Param Set: 25,	AP: 0.20,	AR: 0.94,	F1: 0.32,	FR: 22.61
 ```